In [1]:
import pandas as pd 
import numpy as np 

## The following code is used for gradient descent estimation of the Pig Species Score following modelling using Translating Time by Clancy et al (2001)

### Step 1: Load data frame 1 (the original data excel sheet).

In [2]:
df = pd.read_csv("trans_time_pig.csv")
df

,E_scale,S_scale,Event,PCD_lr,CI
0,0.903,2.226826,Cranial motor nuclei peak,27.290,2
1,1.128,2.343966,Red nucleus peak,36.620,2
2,1.163,2.006812,External capsule appears,28.223,3
3,1.244,1.997811,Cortical layer VI start,30.000,3
4,1.248,2.316166,Mammillo-thalamic tract appears,39.730,3
5,1.250,2.398578,Axons in optic stalk,42.840,2
6,1.272,2.969614,Globus pallidus peak,73.940,3
7,1.302,1.827826,Medial forebrain bundle appears,27.290,1
8,1.305,1.768896,Internal Capsule Appears - starts,26.046,3
9,1.401,2.325416,Superior colliculi peak,45.950,1


### Step 2: Create new data frames 2 & 3 populated with event scores and PCD_lr where Cl (confidence level) > 1 and Cl > 2 

In [52]:
df2 = df[df['CI'] > 1]
df3 = df[df['CI'] > 2]


,E_scale,S_scale,Event,PCD_lr,CI
2,1.163,2.006812,External capsule appears,28.223,3
3,1.244,1.997811,Cortical layer VI start,30.000,3
4,1.248,2.316166,Mammillo-thalamic tract appears,39.730,3
6,1.272,2.969614,Globus pallidus peak,73.940,3
8,1.305,1.768896,Internal Capsule Appears - starts,26.046,3
12,1.422,1.819811,Cortical layer V start,30.000,3
14,1.519,2.045166,Optic fibers travel into superior colliculus,39.730,3
15,1.573,1.797394,Fornix appears,33.510,3
16,1.596,2.223469,Cortical layer VI end,50.000,3
17,1.607,2.212469,Cortical layer IV start,50.000,3


### Step 3: Define following functions: predicted PCD function, Loss function (Sum of Residual Squares), & gradient function 

In [12]:
def predPCD(E_scale, S_scale):
    pcd = np.exp(E_scale + S_scale) + 4.42 
    return pcd

def loss(E, S, PCD_lr):
    loss = np.sum((predPCD(E, S) - PCD_lr) ** 2)
    return loss

def grad(E, S, PCD_lr):
    grad = np.sum(-2 * (predPCD(E, S) - PCD_lr) * np.exp(E + S))
    return grad


### Step 4: Define gradient descent function. 
Within this function you are setting learning rate and convergence criteria (maximum number of iterations and tolerance value).
Next, you set your initial species score value and set a for loop.
Within the for loop we are going through each 'i' iteration within our range of max iterations (10,000).
Within each iteration we calculate our loss and gradient for that specific species score and calculate the species score for the next iteration.
We print a statement with the species score and loss for that iteration.
An if statement is included to detect convergence explained below (before if statement).
If we don't reach tolerance convergence the calculated species score becomes the new species score and we go through another iteration.

In [47]:
def grad_des(E_scale, PCD_lr, lr = 1e-5, max_iter = 10000, tolerance = 1e-6):
    # initializing the first species score
    Si = 3 
    # creating a for loop for the gradient descent to go through 10,000 iterations
    for i in range(max_iter):
        cur_loss = loss(E_scale, Si, PCD_lr)
        gradi = grad(E_scale, Si, PCD_lr)
        Sp =  Si + (gradi * lr)
        '''
        creating an if statement so that if the absolute difference between previous S_score and new S_score 
        is less than our set tolerance (0.000001) we break the for loop
        '''
        print(f'Iter: {i}, Current species scale is: {Si}, and Current SSR(loss) is: {cur_loss}. Yay!! :-) ')
        if np.abs(Si - Sp) < tolerance:
            break

        Si = Sp
    return cur_loss, Si

### Step 5: Define variables from data frames 1, 2, & 3.

In [53]:
event_score = np.array(list(df['E_scale']))
true_PCD = np.array(list(df['PCD_lr']))

event_score2 = np.array(list(df2['E_scale']))
true_PCD2 = np.array(list(df2['PCD_lr']))

event_score3 = np.array(list(df3['E_scale']))
true_PCD3 = np.array(list(df3['PCD_lr']))

### Step 6: Run gradient descent for data frames 1, 2, & 3.

In [54]:
fn_loss, fn_S = grad_des(event_score, true_PCD)
fn_loss, fn_S

Iter: 0, Current species scale is: 3, and Current SSR(loss) is: 94639.42195762935. Yay!! :-) 
Iter: 1, Current species scale is: -0.16207293247298482, and Current SSR(loss) is: 52275.78501917941. Yay!! :-) 
Iter: 2, Current species scale is: -0.06565506083743956, and Current SSR(loss) is: 51305.06712298408. Yay!! :-) 
Iter: 3, Current species scale is: 0.03938730442289101, and Current SSR(loss) is: 50149.21531299252. Yay!! :-) 
Iter: 4, Current species scale is: 0.15454498301534997, and Current SSR(loss) is: 48755.057580771. Yay!! :-) 
Iter: 5, Current species scale is: 0.28167159308622103, and Current SSR(loss) is: 47049.34098908655. Yay!! :-) 
Iter: 6, Current species scale is: 0.42307933296250655, and Current SSR(loss) is: 44929.96780478408. Yay!! :-) 
Iter: 7, Current species scale is: 0.5816391234005915, and Current SSR(loss) is: 42253.99958996479. Yay!! :-) 
Iter: 8, Current species scale is: 0.7608361066008769, and Current SSR(loss) is: 38823.92001226542. Yay!! :-) 
Iter: 9, Cur

(6620.946806112632, 2.1864608077952754)

In [55]:
fn_loss2, fn_S2 = grad_des(event_score2, true_PCD2)
fn_loss2, fn_S2

Iter: 0, Current species scale is: 3, and Current SSR(loss) is: 79452.29381302687. Yay!! :-) 
Iter: 1, Current species scale is: 0.3186678568790198, and Current SSR(loss) is: 42356.24100961829. Yay!! :-) 
Iter: 2, Current species scale is: 0.4477288758576801, and Current SSR(loss) is: 40600.10934652022. Yay!! :-) 
Iter: 3, Current species scale is: 0.5909671767047256, and Current SSR(loss) is: 38429.450793532174. Yay!! :-) 
Iter: 4, Current species scale is: 0.7509776436451805, and Current SSR(loss) is: 35712.54524870897. Yay!! :-) 
Iter: 5, Current species scale is: 0.9306859353941775, and Current SSR(loss) is: 32280.392488954833. Yay!! :-) 
Iter: 6, Current species scale is: 1.132917078617526, and Current SSR(loss) is: 27944.080581621303. Yay!! :-) 
Iter: 7, Current species scale is: 1.359030462492889, and Current SSR(loss) is: 22584.148975166816. Yay!! :-) 
Iter: 8, Current species scale is: 1.6052579829540348, and Current SSR(loss) is: 16427.177689757213. Yay!! :-) 
Iter: 9, Curren

(6117.263716103182, 2.2081270089287015)

In [56]:
fn_loss3, fn_S3 = grad_des(event_score3, true_PCD3)
fn_loss3, fn_S3

Iter: 0, Current species scale is: 3, and Current SSR(loss) is: 73493.53874868067. Yay!! :-) 
Iter: 1, Current species scale is: 0.5204025433231214, and Current SSR(loss) is: 28230.35070846511. Yay!! :-) 
Iter: 2, Current species scale is: 0.6453607828501835, and Current SSR(loss) is: 26594.088626739936. Yay!! :-) 
Iter: 3, Current species scale is: 0.7823631923356172, and Current SSR(loss) is: 24624.57218256488. Yay!! :-) 
Iter: 4, Current species scale is: 0.9329049242421397, and Current SSR(loss) is: 22246.60666016677. Yay!! :-) 
Iter: 5, Current species scale is: 1.098202782547775, and Current SSR(loss) is: 19387.60409172696. Yay!! :-) 
Iter: 6, Current species scale is: 1.278492169774294, and Current SSR(loss) is: 16014.284627341618. Yay!! :-) 
Iter: 7, Current species scale is: 1.4715420329878806, and Current SSR(loss) is: 12217.56320306221. Yay!! :-) 
Iter: 8, Current species scale is: 1.6699125619887707, and Current SSR(loss) is: 8355.730175079396. Yay!! :-) 
Iter: 9, Current s

(2565.7198360518855, 2.15115318237061)